In [ ]:
import sys
import os
import time
import pymongo
import urllib
import pandas as pd
start_time = time.time()
sys.path.append(os.environ.get("PYTHONPATH"))

try:
    import createNewData.data.config as in_config
    from createNewData.pypackages.Azure import Azure
    from createNewData.pypackages.urlHandler import UrlHandler
    AzurePackage = Azure(in_config)
    Url = UrlHandler(in_config)
    
    
except ImportError as e:
    print("Failed to import critical modules for this script.")
    print("Please confirm that files exist in the correct locations.")
    print(e)

# Upload to SQL Database
The below can upload an entire table from a txt or csv file to the Azure SQL Database.
Required fields:
* "UploadToSQL" is the class name for uploading tables to the database.
* tablepath is the path the file is located in.
* tablename is the name of the new table to be created.

In [ ]:
df = pd.read_csv(in_config.shapes)

SqlDataCursor = AzurePackage("UploadToSQL",
                              df,
                              "shapes")

# Return All Unique Shape Id's From SQL Database
The below collects all unique values in a specific column of a database.
Required fields:
* "SelectDistinct" is the class name for collecting the unique values.
* shape_id is the column name.
* [[dbo].[shapes]] is the name of the table to be queried.

In [1]:
shapeIds = AzurePackage("SelectDistinct",
                              "shape_id",
                              "[dbo].[shapes]")

# Collect Elevation Data and Upload to MongoDB
The below collects the longtitude and lattitude for elevation data from the shapes table.
It then generates and send the request to the open elevations URL.
Finally it stores this data to the mongoDB database in Azure Cosmos.

In [2]:
for each in shapeIds.iterrows():
    shapeData = {}
    try:
        # Generate the Pandas table of all the Longtitudes and Latitudes
        # for each shape
        elevations = AzurePackage("SelectLongLat",
                                "[shape_id],[shape_pt_lat],[shape_pt_lon]",
                                "[dbo].[shapes]",
                                "[shape_id]",
                                each[1][0])

        # Generate the Json document for upload to MongoDB
        shapeData[each[1][0].replace(".","_")] = Url("mineElevationData",elevations)
    except urllib.error.HTTPError as e:
        print(in_config.URLOOD)
        print(e)

    try:
        # Upload the Json document to MongoDB
        if not len(shapeData) == 0:
            AzurePackage("UploadToMongo","shapes",shapeData)
        else:
            raise Exception(in_config.NDIDF)
    except TypeError as e:
        print(in_config.TEC)
    except pymongo.errors.DuplicateKeyError as e:
        print(in_config.FIDB)
    except Exception as e:
        print(in_config.UNKMGO)
        print(e)

    print("%s seconds" % (time.time() - start_time))
print("%s seconds" % (time.time() - start_time))



sl=true&retrywrites=false&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@jamescookland@
1242.7248859405518 seconds
mongodb://jamescookland:iRP78YZTLnrKTHSTKB4cXmHPA4Xf3RMeMLzXvZsNjimcgXNKEkysaUlpk3YjuBQxAIMZc7nRjtTvhCgW0fjxfw==@jamescookland.mongo.cosmos.azure.com:10255/?ssl=true&retrywrites=false&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@jamescookland@
1243.8572895526886 seconds
mongodb://jamescookland:iRP78YZTLnrKTHSTKB4cXmHPA4Xf3RMeMLzXvZsNjimcgXNKEkysaUlpk3YjuBQxAIMZc7nRjtTvhCgW0fjxfw==@jamescookland.mongo.cosmos.azure.com:10255/?ssl=true&retrywrites=false&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@jamescookland@
1245.2728130817413 seconds
mongodb://jamescookland:iRP78YZTLnrKTHSTKB4cXmHPA4Xf3RMeMLzXvZsNjimcgXNKEkysaUlpk3YjuBQxAIMZc7nRjtTvhCgW0fjxfw==@jamescookland.mongo.cosmos.azure.com:10255/?ssl=true&retrywrites=false&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@jamescookland@
1247.1386604309082 seconds
mongodb://jamescookland:iRP78YZTLnrKTHSTKB4cXmHPA4Xf3R

# Read Data From MongoDB and write to SQL Database

In [1]:
listOfObjects = []
listOfElevations = []

dbcollections = AzurePackage("SelectFromMongo")
for collection in dbcollections.find():
    for each in collection:
        if type(collection[each]) is dict:
            listOfObjects.append(collection[each])

for each in listOfObjects:
    for key, value in each.items():
        for elevation in value:
            listOfElevations.append(elevation)

df = pd.DataFrame(listOfElevations)
dfTrimmed = df.drop_duplicates()


mongodb://jamescookland:iRP78YZTLnrKTHSTKB4cXmHPA4Xf3RMeMLzXvZsNjimcgXNKEkysaUlpk3YjuBQxAIMZc7nRjtTvhCgW0fjxfw==@jamescookland.mongo.cosmos.azure.com:10255/?ssl=true&retrywrites=false&replicaSet=globaldb&maxIdleTimeMS=120000&appName=@jamescookland@


In [3]:
df = pd.read_csv(in_config.shapes)
SqlDataCursor = AzurePackage("UploadToSQL",
                              dfTrimmed,
                              "elevations")

AttributeError: 'str' object has no attribute 'close'